## 准备数据

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 定义第一个全连接层的权重和偏置
        # 输入展平后为 28*28 = 784 维，输出维度为 512
        self.W1 = tf.Variable(
            initial_value=tf.random.uniform([28*28, 512], minval=-0.1, maxval=0.1),
            dtype=tf.float32
        )
        self.b1 = tf.Variable(
            initial_value=tf.zeros([512]),
            dtype=tf.float32
        )
        # 定义第二个全连接层的权重和偏置
        # 隐藏层输出维度为 512，输出层对应 10 个类别
        self.W2 = tf.Variable(
            initial_value=tf.random.uniform([512, 10], minval=-0.1, maxval=0.1),
            dtype=tf.float32
        )
        self.b2 = tf.Variable(
            initial_value=tf.zeros([10]),
            dtype=tf.float32
        )

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 将输入 x 展平为二维张量 [batch_size, 784]
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        
        # 计算第一个全连接层的输出，并使用 ReLU 激活函数
        h1 = tf.nn.relu(tf.matmul(flat_x, self.W1) + self.b1)
        
        # 计算输出层 logits，不使用激活函数（返回未归一化的值）
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.495654 ; accuracy 0.0805
epoch 1 : loss 2.4653149 ; accuracy 0.08241667
epoch 2 : loss 2.4375677 ; accuracy 0.08505
epoch 3 : loss 2.4120104 ; accuracy 0.08755
epoch 4 : loss 2.3883152 ; accuracy 0.0913
epoch 5 : loss 2.3662112 ; accuracy 0.09565
epoch 6 : loss 2.345474 ; accuracy 0.10185
epoch 7 : loss 2.325919 ; accuracy 0.109516665
epoch 8 : loss 2.307389 ; accuracy 0.11731666
epoch 9 : loss 2.2897506 ; accuracy 0.12593333
epoch 10 : loss 2.2728932 ; accuracy 0.1356
epoch 11 : loss 2.2567225 ; accuracy 0.14576666
epoch 12 : loss 2.2411551 ; accuracy 0.15778333
epoch 13 : loss 2.2261221 ; accuracy 0.1702
epoch 14 : loss 2.2115636 ; accuracy 0.18245
epoch 15 : loss 2.1974282 ; accuracy 0.19521667
epoch 16 : loss 2.183671 ; accuracy 0.2073
epoch 17 : loss 2.1702523 ; accuracy 0.22033334
epoch 18 : loss 2.1571403 ; accuracy 0.23386666
epoch 19 : loss 2.1443064 ; accuracy 0.24685
epoch 20 : loss 2.1317253 ; accuracy 0.25916666
epoch 21 : loss 2.1193745 ; accuracy 0.27178